In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
X_1 = pd.read_csv('amazon_cells_labelled.txt', sep="\t", usecols=[0], header=None)
Y_1 = pd.read_csv('amazon_cells_labelled.txt', sep="\t", usecols=[1], header=None)

X_2 = pd.read_csv('imdb_labelled.txt', sep="\t", usecols=[0], header=None)
Y_2 = pd.read_csv('imdb_labelled.txt', sep="\t", usecols=[1], header=None)

X_3 = pd.read_csv('yelp_labelled.txt', sep="\t", usecols=[0], header=None)
Y_3 = pd.read_csv('yelp_labelled.txt', sep="\t", usecols=[1], header=None)


X = np.concatenate((X_1.values[:,0], X_2.values[:,0]), axis=0)
Y = np.concatenate((Y_1.values[:,0], Y_2.values[:,0]), axis=0)
X = np.concatenate((X, X_3.values[:,0]), axis=0)
Y = np.concatenate((Y, Y_3.values[:,0]), axis=0)
#X = X_1.values[:,0]
#Y = Y_1.values[:,0]

len(Y)

#print(Y)

2748

### Creating Dictionary

In [3]:
# Treating Sentences
allSentences = []
charsSplit = "\\`*_{}[]()>#+-.!$,:&?"
charsRemove = ".,-_;\"\'"
for x in X :
    for c in charsSplit:
        x = x.replace(c, ' '+ c +' ')
    for c in charsRemove:
        x = x.replace(c, '')
    allSentences.append(x.lower() )

In [4]:
allWords = []
for x in allSentences :
    allWords.extend(x.split(" "))


allWords = list(set(allWords))
allWords.sort()


In [10]:
#len(Y)

positiveCount = [0] * len(allWords)
negativeCount = [0] * len(allWords)

sentenceNumber = 0
for sentenceNumber, sentence in enumerate(allSentences) :
    for word in sentence.split(" "):
        wordIndex = allWords.index(word)
        if (Y[sentenceNumber] == 1):
            positiveCount[wordIndex] = positiveCount[wordIndex] + 1
        else:
            negativeCount[wordIndex] = negativeCount[wordIndex] + 1
            
    #sentenceNumber = sentenceNumber + 1
    
allCount =  np.array(positiveCount) + np.array(negativeCount)
probPositive = np.divide(positiveCount, allCount)

#print(len(X), " ", len(Y), " ", len(probPositive), " ", len(allWords))
len(allWords)
allWordsFiltered =  np.array(allWords)

len(allWordsFiltered)
index_to_remove = []
for index, prob in enumerate(probPositive):
    if not((prob <= 0.48) or (prob >= 0.52)):
        #print(index , "-- " , prob, "-- " )
        #allWordsFiltered = np.delete(allWordsFiltered, index)
        index_to_remove.append(index)
        a = 1

allWordsFiltered = np.delete(allWordsFiltered, index_to_remove)

print(len(X), " ", len(Y), " ", len(probPositive), " ", len(allWordsFiltered))

#print(allWords)     

#print(probPositive)

#probNegative = np.divide(negativeCount, allCount)
#print(probNegative)

2748   2748   5372   4929


In [15]:
allWords = np.array(allWordsFiltered).tolist()
X = []
for x in allSentences :
    sentenceV = [0] * len(allWords)
    words = x.split(" ")
    for w in words :
        try:
            index = allWords.index(w)
            sentenceV[index] = 1 #sentenceV[index] + 1
        except ValueError:
            pass

    X.append(sentenceV)

len(allWords)
len(X)

2748

In [16]:
#X[2][allWords.index('great')]

In [17]:
from sklearn.cross_validation import cross_val_score
cross_val_k = 10


# from sklearn import svm
# clf = svm.SVC(kernel='linear', C=1.0)

# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()


# from sklearn.naive_bayes import BernoulliNB
# clf = BernoulliNB()

# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB()


# from sklearn.decomposition import KernelPCA
# kpca = KernelPCA(n_components=300, kernel='linear')
# X_KPCA = kpca.fit_transform(X)

#print(len(X_KPCA))
#print(len(X))

from sklearn.naive_bayes import MultinomialNB
clf1 = MultinomialNB()

from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier(n_neighbors=5)

# from sklearn import tree
# clf2 = tree.DecisionTreeClassifier()
# from sklearn.ensemble import RandomForestClassifier
# clf2 = RandomForestClassifier(n_estimators=10)

from sklearn.linear_model import LogisticRegression
clf3 = LogisticRegression(penalty='l2', C=1.0)
# from sklearn import svm
# clf3 = svm.SVC(kernel='linear', C=1.0, probability=True)

from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)], voting='soft', weights=[3,2,2])

accuracy = cross_val_score(eclf, X, Y, cv=cross_val_k, scoring='accuracy').mean()
print('MultinomialNB', accuracy)

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


MultinomialNB 0.852974803574


Melhor resultado:

- VotingClassifier

MultinomialNB (peso 3)
KNeighborsClassifier (peso 2)    -> 0.69429994326
LogisticRegression (peso 2) penalty='l2'
0.834031245492

